# SAM 3 Segmentation for 3DGS (Official API)

**Based on**: Official SAM 3 examples from Facebook Research

**Features**:
- ✅ Text prompts ("green frog", "pink flower")
- ✅ Batched inference for speed
- ✅ Automatic confidence filtering
- ✅ ZIP or Video input

**Source**: `sam3_image_predictor_example.ipynb` + `sam3_image_batched_inference.ipynb`

---

# Part 1: Installation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("✅ Drive mounted")

In [ ]:
print("🔧 Installing SAM 3 (fixed method)...\n")

# Cleanup
print("[1/6] Cleanup...")
!pip uninstall -y -q jax jaxlib pytensor shap music21 2>/dev/null || true
!pip uninstall -y -q opencv-python opencv-contrib-python opencv-python-headless 2>/dev/null || true
!pip uninstall -y -q numpy 2>/dev/null || true

print("[2/6] NumPy 1.26...")
!pip install -q numpy==1.26.0

print("[3/6] Core dependencies...")
!pip install -q timm>=1.0.17 tqdm ftfy==6.1.1 regex iopath>=0.1.10 typing_extensions huggingface_hub
!pip install -q opencv-python==4.9.0.80 matplotlib pillow scikit-learn jupyter ipywidgets

print("[4/6] Clone SAM 3...")
import os
if not os.path.exists('/content/sam3'):
    !git clone -q https://github.com/facebookresearch/sam3.git
else:
    print("   Already cloned")

print("[5/6] Install SAM 3...")
%cd /content/sam3
!pip install -q -e ".[notebooks]"
%cd /content

print("[6/6] Python path...")
import sys
if '/content/sam3' not in sys.path:
    sys.path.insert(0, '/content/sam3')

print("\n" + "="*60)
print("✅ INSTALLATION COMPLETE")
print("="*60)
print("\n⚠️  RESTART RUNTIME NOW!")
print("="*60)

---
# ⚠️ RESTART RUNTIME!
Then continue to Part 2 ⬇️
---

# Part 2: Setup (After Restart)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("✅ Drive mounted")

In [ ]:
print("🔍 Verifying installation...\n")

import sys
import os

# CRITICAL: Add BOTH paths
sam3_repo = '/content/sam3'
if sam3_repo not in sys.path:
    sys.path.insert(0, sam3_repo)

# Force reimport if already imported incorrectly
if 'sam3' in sys.modules:
    del sys.modules['sam3']

# Now import
import torch
import numpy as np
import cv2
from PIL import Image

print("Importing SAM 3 modules...")

# Import from the correct location
from sam3.model_builder import build_sam3_image_model  # ✅ Correct import!
from sam3.model.sam3_image_processor import Sam3Processor
from sam3.visualization_utils import plot_results

# Set sam3_root
sam3_root = '/content/sam3'

print("✅ All imports successful!\n")
print("📊 Environment:")
print(f"   Python: {sys.version.split()[0]}")
print(f"   PyTorch: {torch.__version__}")
print(f"   NumPy: {np.__version__}")
print(f"   OpenCV: {cv2.__version__}")
print(f"   GPU: {torch.cuda.get_device_name(0)}")
print(f"   SAM 3 root: {sam3_root}")

if np.__version__.startswith('1.26'):
    print("\n🎉 Ready for SAM 3!")
else:
    print(f"\n⚠️ NumPy: {np.__version__} (expected 1.26.x)")

In [ ]:
# PyTorch optimization (from official example)
import torch

# Turn on tfloat32 for Ampere GPUs
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

# Use bfloat16 (if your GPU supports it, T4 does)
torch.autocast("cuda", dtype=torch.bfloat16).__enter__()

# Inference mode
torch.inference_mode().__enter__()

print("✅ PyTorch optimizations enabled")

In [ ]:
from huggingface_hub import login

print("🔐 Hugging Face Login")
print("💡 Token: https://huggingface.co/settings/tokens\n")

login()

print("\n✅ Logged in!")

## Summary

In [ ]:
# ============================================
# SAM 3 Video Segmentation - Standalone (COLMAP용)
# ============================================

import torch
import numpy as np
from PIL import Image
import cv2
from pathlib import Path
from tqdm import tqdm

# SAM 3 imports
from sam3.model_builder import build_sam3_image_model
from sam3.model.sam3_image_processor import Sam3Processor

# ============================================
# CONFIGURATION - 여기만 수정하세요!
# ============================================

VIDEO_PATH = "/content/drive/MyDrive/25_sch/CV/Ass3/raw_data/IMG_7593.MOV"  # 비디오 경로
OUTPUT_DIR = "/content/ydino_0.4"  # 저장 경로
TEXT_PROMPTS = ["yellow toy"]  # 세그멘테이션할 객체

# Optional settings
SKIP_SECONDS = 4  # 처음 N초 건너뛰기 (학번 제외)
SAMPLE_RATE = 6   # 1 프레임 / N 프레임 (6 = 매 6번째 프레임)
CONFIDENCE_THRESHOLD = 0.4  # 0.1-0.3 추천

# ============================================
# 1. Load Model
# ============================================

print("🤖 Loading SAM 3 model...\n")

device = "cuda" if torch.cuda.is_available() else "cpu"
model = build_sam3_image_model().to(device)
processor = Sam3Processor(model, confidence_threshold=CONFIDENCE_THRESHOLD)

print(f"✅ Model loaded on {device}")
print(f"   GPU Memory: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB\n")

# ============================================
# 2. Process Video
# ============================================

print(f"🎬 Processing video: {Path(VIDEO_PATH).name}\n")
print(f"   Output: {OUTPUT_DIR}")
print(f"   Prompts: {TEXT_PROMPTS}")
print(f"   Skip: {SKIP_SECONDS}s, Sample: 1/{SAMPLE_RATE}\n")

# Create output directory
output_path = Path(OUTPUT_DIR)
output_path.mkdir(parents=True, exist_ok=True)

# Open video
cap = cv2.VideoCapture(VIDEO_PATH)

if not cap.isOpened():
    raise ValueError(f"Cannot open video: {VIDEO_PATH}")

# Video info
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
skip_frames = int(SKIP_SECONDS * fps)

print(f"📊 Video info:")
print(f"   FPS: {fps:.1f}")
print(f"   Total frames: {total_frames}")
print(f"   Skip frames: {skip_frames}")

expected = (total_frames - skip_frames) // SAMPLE_RATE
print(f"   Expected output: ~{expected} frames\n")

# Process frames
frame_idx = 0
output_idx = 0
processed = 0
failed = 0

pbar = tqdm(total=expected, desc="Processing")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Skip initial frames (student ID)
    if frame_idx < skip_frames:
        frame_idx += 1
        continue

    # Sample frames
    if (frame_idx - skip_frames) % SAMPLE_RATE != 0:
        frame_idx += 1
        continue

    try:
        # Convert BGR to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        height, width = frame_rgb.shape[:2]

        # Convert to PIL
        image_pil = Image.fromarray(frame_rgb)

        # Set image (extract features)
        inference_state = processor.set_image(image_pil)

        # Initialize combined mask
        combined_mask = np.zeros((height, width), dtype=bool)

        # Try each text prompt
        for prompt in TEXT_PROMPTS:
            try:
                # Reset previous prompts
                processor.reset_all_prompts(inference_state)

                # Set text prompt (PCS)
                output = processor.set_text_prompt(
                    state=inference_state,
                    prompt=prompt
                )

                # Extract masks
                if "masks" in output:
                    masks = output["masks"]

                    # Convert tensor to list
                    if torch.is_tensor(masks):
                        masks = [masks[i] for i in range(len(masks))]

                    # Process each mask
                    for mask in masks:
                        # Tensor to numpy
                        if torch.is_tensor(mask):
                            mask = mask.cpu().numpy()

                        # Convert to boolean
                        if mask.dtype != bool:
                            mask = mask > 0.5

                        # 3D → 2D
                        while mask.ndim > 2:
                            mask = mask[0]

                        # Resize if needed
                        if mask.shape != (height, width):
                            mask = cv2.resize(
                                mask.astype(np.uint8),
                                (width, height),
                                interpolation=cv2.INTER_NEAREST
                            ).astype(bool)

                        # Combine masks (OR operation)
                        combined_mask |= mask

            except Exception as e:
                # Skip failed prompt
                pass

        # Fallback: if no mask found, keep full image
        if not combined_mask.any():
            combined_mask = np.ones((height, width), dtype=bool)

        # Create RGB with BLACK background (for COLMAP)
        rgb_with_black_bg = np.zeros((height, width, 3), dtype=np.uint8)

        # Copy only masked region from original frame
        rgb_with_black_bg[combined_mask] = frame_rgb[combined_mask]

        # Background is black (0,0,0) by default

        # Save as PNG (3-channel RGB, not RGBA)
        out_file = output_path / f"frame_{output_idx:04d}.png"
        cv2.imwrite(str(out_file), cv2.cvtColor(rgb_with_black_bg, cv2.COLOR_RGB2BGR))
        # ============================================

        processed += 1
        output_idx += 1
        pbar.update(1)

    except Exception as e:
        failed += 1

    frame_idx += 1

cap.release()
pbar.close()

# ============================================
# 3. Summary
# ============================================

print(f"\n{'='*60}")
print(f"✅ PROCESSING COMPLETE")
print(f"{'='*60}")
print(f"\n📊 Results:")
print(f"   Processed: {processed} frames")
print(f"   Failed: {failed} frames")
print(f"   Coverage: {processed / expected * 100:.1f}%")
print(f"\n📁 Output directory: {output_path}")
print(f"\n🎯 Files created:")

# List first 5 files
files = sorted(list(output_path.glob("frame_*.png")))
for f in files[:5]:
    print(f"   {f.name}")
if len(files) > 5:
    print(f"   ... and {len(files) - 5} more")

print(f"\n✅ Total: {len(files)} PNG files with BLACK background (COLMAP ready)")

🤖 Loading SAM 3 model...

✅ Model loaded on cuda
   GPU Memory: 7.08 GB

🎬 Processing video: IMG_7593.MOV

   Output: /content/ydino_0.4
   Prompts: ['yellow toy']
   Skip: 4s, Sample: 1/6

📊 Video info:
   FPS: 30.0
   Total frames: 1007
   Skip frames: 119
   Expected output: ~148 frames




Processing:  11%|█▏        | 17/148 [01:50<14:11,  6.50s/it]

Processing:  83%|████████▎ | 123/148 [05:29<01:04,  2.59s/it]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ============================================
# 4. Create ZIP File
# ============================================

import zipfile
import shutil

print(f"\n{'='*60}")
print(f"📦 Creating ZIP file...")
print(f"{'='*60}\n")

# ZIP file path
zip_name = f"{Path(VIDEO_PATH).stem}_dino_0.2"
zip_file_name = f"{zip_name}.zip"
zip_path = output_path.parent / zip_file_name

print(f"   Output: {zip_path.name}")

# Get all PNG files
png_files = sorted(list(output_path.glob("frame_*.png")))

if not png_files:
    print("   ❌ No PNG files found!")
else:
    # Create ZIP
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for png_file in tqdm(png_files, desc="Zipping"):
            # Add to ZIP with relative path
            zipf.write(png_file, f"{zip_name}/{png_file.name}")

    # Print results
    zip_size_mb = zip_path.stat().st_size / (1024 * 1024)
    print(f"\n✅ ZIP created successfully!")
    print(f"   Files: {len(png_files)} frames")
    print(f"   Size: {zip_size_mb:.1f} MB")
    print(f"   Path: {zip_path}")

# ============================================
# 5. Copy to Drive (Optional)
# ============================================

SAVE_TO_DRIVE = True  # True로 설정하면 Drive에 복사
DRIVE_PATH = "/content/drive/MyDrive/25_sch/CV/Ass3/segmented_outputs"

if SAVE_TO_DRIVE:
    print(f"\n📤 Copying to Google Drive...")

    drive_dest = Path(DRIVE_PATH)
    drive_dest.mkdir(parents=True, exist_ok=True)

    drive_file = drive_dest / zip_path.name
    shutil.copy(zip_path, drive_file)

    print(f"   ✅ Saved to: {drive_file}")
else:
    print(f"\n⏭️  Skipping Drive upload (SAVE_TO_DRIVE=False)")

print(f"\n{'='*60}")
print(f"🎉 ALL DONE!")
print(f"{'='*60}")

In [ ]:
# ============================================
# SAM 3 Batch Video Segmentation - COLMAP용
# ============================================

import torch
import numpy as np
from PIL import Image
import cv2
from pathlib import Path
from tqdm import tqdm
import zipfile
import shutil

# SAM 3 imports
from sam3.model_builder import build_sam3_image_model
from sam3.model.sam3_image_processor import Sam3Processor

# ============================================
# BATCH CONFIGURATION - 여기만 수정하세요!
# ============================================

# 비디오 설정 리스트
VIDEO_CONFIGS = [
    {
        'video_path': "/content/drive/MyDrive/25_sch/CV/Ass3/raw_data/IMG_7502.MOV",
        'output_name': "obj1_chimbread",
        'text_prompts': ["plush with man face"],
        'confidence': 0.3,
        'skip_seconds': 4,
        'sample_rate': 6,
    },
    {
        'video_path': "/content/drive/MyDrive/25_sch/CV/Ass3/raw_data/IMG_7495.MOV",
        'output_name': "obj2_wood_toy",
        'text_prompts': ["wood toy"],
        'confidence': 0.3,
        'skip_seconds': 4,
        'sample_rate': 6,
    },
    {
        'video_path': "/content/drive/MyDrive/25_sch/CV/Ass3/raw_data/IMG_7492.MOV",
        'output_name': "obj3_dragon",
        'text_prompts': ["yellow dino"],
        'confidence': 0.3,
        'skip_seconds': 4,
        'sample_rate': 6,
    },
]

# 전역 설정
BASE_OUTPUT_DIR = "/content/segmented_outputs"
DRIVE_SAVE_PATH = "/content/drive/MyDrive/25_sch/CV/Ass3/segmented_outputs"
SAVE_TO_DRIVE = True

# ============================================
# 1. Load Model (한 번만)
# ============================================

print("="*60)
print("🤖 Loading SAM 3 model...")
print("="*60 + "\n")

device = "cuda" if torch.cuda.is_available() else "cpu"
model = build_sam3_image_model().to(device)

print(f"✅ Model loaded on {device}")
print(f"   GPU Memory: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB\n")

# ============================================
# 2. Process Each Video
# ============================================

all_results = []

for idx, config in enumerate(VIDEO_CONFIGS, 1):
    print("="*60)
    print(f"📹 Video {idx}/{len(VIDEO_CONFIGS)}: {config['output_name']}")
    print("="*60)

    video_path = config['video_path']
    output_name = config['output_name']
    text_prompts = config['text_prompts']
    confidence = config['confidence']
    skip_seconds = config['skip_seconds']
    sample_rate = config['sample_rate']

    # Create processor with specific confidence
    processor = Sam3Processor(model, confidence_threshold=confidence)

    print(f"\n📋 Configuration:")
    print(f"   Video: {Path(video_path).name}")
    print(f"   Prompts: {text_prompts}")
    print(f"   Confidence: {confidence}")
    print(f"   Skip: {skip_seconds}s, Sample: 1/{sample_rate}")

    # Create output directory
    output_dir = Path(BASE_OUTPUT_DIR) / output_name
    output_dir.mkdir(parents=True, exist_ok=True)

    # Open video
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"❌ Cannot open video: {video_path}")
        all_results.append({
            'name': output_name,
            'status': 'failed',
            'error': 'Cannot open video'
        })
        continue

    # Video info
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    skip_frames = int(skip_seconds * fps)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    print(f"\n📊 Video info:")
    print(f"   Resolution: {width}x{height}")
    print(f"   FPS: {fps:.1f}")
    print(f"   Total frames: {total_frames}")
    print(f"   Skip frames: {skip_frames}")

    expected = (total_frames - skip_frames) // sample_rate
    print(f"   Expected output: ~{expected} frames\n")

    # Process frames
    frame_idx = 0
    output_idx = 0
    processed = 0
    failed = 0

    pbar = tqdm(total=expected, desc=f"Processing {output_name}")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Skip initial frames
        if frame_idx < skip_frames:
            frame_idx += 1
            continue

        # Sample frames
        if (frame_idx - skip_frames) % sample_rate != 0:
            frame_idx += 1
            continue

        try:
            # Convert BGR to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image_pil = Image.fromarray(frame_rgb)

            # SAM 3 segmentation
            inference_state = processor.set_image(image_pil)
            combined_mask = np.zeros((height, width), dtype=bool)

            # Try each text prompt
            for prompt in text_prompts:
                try:
                    processor.reset_all_prompts(inference_state)
                    output = processor.set_text_prompt(
                        state=inference_state,
                        prompt=prompt
                    )

                    if "masks" in output:
                        masks = output["masks"]

                        if torch.is_tensor(masks):
                            masks = [masks[i] for i in range(len(masks))]

                        for mask in masks:
                            if torch.is_tensor(mask):
                                mask = mask.cpu().numpy()

                            if mask.dtype != bool:
                                mask = mask > 0.5

                            while mask.ndim > 2:
                                mask = mask[0]

                            if mask.shape != (height, width):
                                mask = cv2.resize(
                                    mask.astype(np.uint8),
                                    (width, height),
                                    interpolation=cv2.INTER_NEAREST
                                ).astype(bool)

                            combined_mask |= mask

                except Exception as e:
                    pass

            # Fallback: keep full image if no mask
            if not combined_mask.any():
                combined_mask = np.ones((height, width), dtype=bool)

            # Create RGB with black background
            rgb_with_black_bg = np.zeros((height, width, 3), dtype=np.uint8)
            rgb_with_black_bg[combined_mask] = frame_rgb[combined_mask]

            # Save as PNG
            out_file = output_dir / f"frame_{output_idx:04d}.png"
            cv2.imwrite(str(out_file), cv2.cvtColor(rgb_with_black_bg, cv2.COLOR_RGB2BGR))

            processed += 1
            output_idx += 1
            pbar.update(1)

        except Exception as e:
            failed += 1

        frame_idx += 1

    cap.release()
    pbar.close()

    # ============================================
    # 3. Create ZIP for this video
    # ============================================

    print(f"\n📦 Creating ZIP file...")

    zip_name = f"{output_name}_segmented_c{confidence}.zip"
    zip_path = Path(BASE_OUTPUT_DIR) / zip_name

    png_files = sorted(list(output_dir.glob("frame_*.png")))

    if png_files:
        with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for png_file in tqdm(png_files, desc="Zipping", leave=False):
                zipf.write(png_file, f"{output_name}/images/{png_file.name}")

        zip_size_mb = zip_path.stat().st_size / (1024 * 1024)

        print(f"   ✅ ZIP created: {zip_name}")
        print(f"   Files: {len(png_files)} frames")
        print(f"   Size: {zip_size_mb:.1f} MB")

        # Save to Drive
        if SAVE_TO_DRIVE:
            drive_dest = Path(DRIVE_SAVE_PATH)
            drive_dest.mkdir(parents=True, exist_ok=True)
            drive_file = drive_dest / zip_name
            shutil.copy(zip_path, drive_file)
            print(f"   📤 Saved to Drive: {drive_file.name}")

        # Store results
        all_results.append({
            'name': output_name,
            'status': 'success',
            'video': Path(video_path).name,
            'frames': processed,
            'failed': failed,
            'prompts': text_prompts,
            'confidence': confidence,
            'zip_path': str(zip_path),
            'zip_size_mb': zip_size_mb
        })
    else:
        print(f"   ❌ No frames extracted!")
        all_results.append({
            'name': output_name,
            'status': 'failed',
            'error': 'No frames extracted'
        })

    print()

# ============================================
# 4. Final Summary
# ============================================

print("="*60)
print("🎉 BATCH PROCESSING COMPLETE!")
print("="*60)

print(f"\n📊 Summary:")
print(f"   Total videos: {len(VIDEO_CONFIGS)}")
print(f"   Successful: {sum(1 for r in all_results if r['status'] == 'success')}")
print(f"   Failed: {sum(1 for r in all_results if r['status'] == 'failed')}")

print(f"\n📋 Results:\n")

for result in all_results:
    if result['status'] == 'success':
        print(f"✅ {result['name']}")
        print(f"   Video: {result['video']}")
        print(f"   Frames: {result['frames']} (failed: {result['failed']})")
        print(f"   Prompts: {result['prompts']}")
        print(f"   Confidence: {result['confidence']}")
        print(f"   ZIP: {Path(result['zip_path']).name} ({result['zip_size_mb']:.1f} MB)")
    else:
        print(f"❌ {result['name']}")
        print(f"   Error: {result.get('error', 'Unknown')}")
    print()

if SAVE_TO_DRIVE:
    print(f"📁 All files saved to: {DRIVE_SAVE_PATH}")

print("="*60)
print("🎯 Next Steps:")
print("   1. Download ZIP files from Drive")
print("   2. Extract to colmap_input/")
print("   3. Run COLMAP on each dataset")
print("   4. Train 3DGS models")
print("="*60)